<a href="https://colab.research.google.com/github/forgef/Query_WTO_API/blob/master/1_Data_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use the WTO API to request data from [WTO Time Series Website](https://timeseries.wto.org/). <br>


Info on how to use the API [here](https://apiportal.wto.org/)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import urllib.parse
import requests
import pandas as pd
import json
from datetime import datetime
import chardet
from concurrent.futures import ThreadPoolExecutor
dataFolder='/content/gdrive/My Drive/LDC/Data/' 
wto_api_key=

In [0]:
# Preferential by detailed HS 6-digit ; HS MFN - Simple average with ad valorem equivalents (AVE) (Percent) 
indicators=['HS_P_0070','HS_A_0015'] ; tariff_names=['Preferential','MFN']
INDICATORS=dict(zip(tariff_names,indicators))
#Reporting Countries
reporting=['036','124','918','352','392','554','578','756','792','840'] ; country_names=['Australia', 'Canada', 'European Union', 'Iceland', 'Japan', 'New Zealand', 'Norway', 'Switzerland', 'Turkey', 'United States']
OECD= dict(zip(country_names,reporting))

In [0]:
###############################################################################################################################################################################
def create_urls(country, tariff, maxrows='1000000'):
  # Create the URL for the request
  wto='https://api.wto.org/timeseries/v1/data?'
  mykey=wto_api_key
  URLs=[]
  ### URLs_pref
  for year in range(1980,2016):
    params = urllib.parse.urlencode({
        # Request parameters
        'i': INDICATORS[tariff], #
        'r': OECD[country],
        'ps': year,
        'pc': 'HS6',
        'max': maxrows,
        'fmt': 'json',
        'mode': 'full',
        'dec': 'default',
        'lang': '1',
        'meta': 'false',
        'off': '0',
        'subscription-key':mykey
    })
    url=''.join([wto,params])
    URLs.append(url)
  return URLs
###############################################################################################################################################################################
def query_wto(url):
  url_year=url.replace('https://api.wto.org/timeseries/v1/data?i='+INDICATORS[tariff]+'&r='+OECD[country]+'&ps=','')
  url_year=url_year.replace('&pc=HS6&max='+maxrows+'&fmt=json&mode=full&dec=default&lang=1&meta=false&off=0&subscription-key='+mykey,'')
  
  ###
  # Get the data
  print(f"Catching the request for {url_year}",end=', ')
  req=requests.get(url)
  req.encoding='ascii'
  print(f"Loads {url_year}",end=', ')

  req=json.loads(req.text) 
  ###############################################################################################################################################################################
  # To Pandas DataFrame
  if tariff=='Preferential':
    try:
      df=pd.json_normalize(req["Dataset"])
      print(f"{''.join(str(df.Year.unique().tolist()))} to pd",end=', ')
      keep=['ReportingEconomyCode','ReportingEconomy','PartnerEconomyCode','PartnerEconomy','ProductOrSectorCode','Year','TextValue','Value']
      rename={'ReportingEconomyCode': 'importerCode',
              'ReportingEconomy':'importer',
              'PartnerEconomyCode':'exporterCode',	
              'PartnerEconomy':'exporter',
              'ProductOrSectorCode':'HS6',
              'Year':'year',
              'TextValue':'preference',
              'Value':'tariff_pct'}
      df=df[keep]        
      df.rename(columns=rename,inplace=True)
      return df
    except KeyError:
        pass
  else:
    try:
      df=pd.json_normalize(req["Dataset"])
      print(f"{''.join(str(df.Year.unique().tolist()))} to pd",end=', ')
      keep=['ReportingEconomyCode','ReportingEconomy','ProductOrSectorCode','Year','TextValue','Value']
      rename={'ReportingEconomyCode': 'importerCode',
        'ReportingEconomy':'importer',
        'ProductOrSectorCode':'HS6',
        'Year':'year',
        'TextValue':'preference',
        'Value':'tariff_pct'}
      df=df[keep]
      df.rename(columns=rename,inplace=True)
      df['preference']='MFN'
      return df
    except KeyError:
        pass


In [85]:
for tariff in reversed(tariff_names):  
  for country in country_names:
    print(f"\nStarted {country} for {tariff}")
    # Create URLs
    URLs=create_urls(country,tariff)
    # Request JSON files and put them in Pandas DataFrames
    DFs=[]
    pool = ThreadPoolExecutor(max_workers=3)
    for f in pool.map(query_wto, URLs):
      DFs.append(f)
    # Concatenate dataframes
    print("Concatenating")

    df=pd.concat(DFs)
    # Save
    df.to_csv(''.join(dataFolder+country+'_'+tariff+'.csv'))
    if tariff=='Preferential':
      print("{} for {} has:\n- {:,} observations. \n- {:,} unique HS6. \n- {:,} unique exporters. \n- {} preferential tariff names.".format(country,tariff,len(df), len(df.HS6.unique()),len(df.exporter.unique()),len(df.preference.unique())))
    else:
      print("{} for {} has:\n- {:,} observations. \n- {:,} unique HS6. \n-For years: {}".format(country,tariff,len(df), len(df.HS6.unique()),df.year.unique())) 

    display(df.head())



Started Australia for MFN
Catching the request for 1980, Catching the request for 1981, Catching the request for 1982, Loads 1981, Catching the request for 1983, Loads 1982, Catching the request for 1984, Loads 1983Loads 1984, Catching the request for 1985, , Catching the request for 1986, Loads 1980, Catching the request for 1987, Loads 1986Loads 1985, , Catching the request for 1988, Catching the request for 1989, Loads 1987, Catching the request for 1990, Loads 1989, Catching the request for 1991, Loads 1990, Catching the request for 1992, Loads 1991, Catching the request for 1993, Loads 1992, Catching the request for 1994, Loads 1988, Catching the request for 1995, Loads 1993, Catching the request for 1996, Loads 1994, Catching the request for 1997, Loads 1995, Catching the request for 1998, Loads 1997, Catching the request for 1999, Loads 1998, Catching the request for 2000, Loads 1999, Catching the request for 2001, Loads 2000, Catching the request for 2002, Loads 2001, Catching

,importerCode,importer,HS6,year,preference,tariff_pct
0,036,Australia,292422,1996,MFN,0.0
1,036,Australia,340600,1996,MFN,5.0
2,036,Australia,292429,1996,MFN,0.0
3,036,Australia,340700,1996,MFN,5.0
4,036,Australia,292511,1996,MFN,0.0



Started Canada for MFN
Catching the request for 1980, Catching the request for 1981Catching the request for 1982, , Loads 1982, Catching the request for 1983, Loads 1983, Catching the request for 1984, Loads 1980, Catching the request for 1985, Loads 1981, Catching the request for 1986, Loads 1985, Catching the request for 1987, Loads 1986, Catching the request for 1988, Loads 1987, Catching the request for 1989, Loads 1984, Catching the request for 1990, Loads 1988, Catching the request for 1991, Loads 1989, Catching the request for 1992, Loads 1990, Catching the request for 1993, Loads 1992, Catching the request for 1994, Loads 1993, Catching the request for 1995, Loads 1991, Catching the request for 1996, Loads 1994, Catching the request for 1997, Loads 1995, Catching the request for 1998, Loads 1996, Catching the request for 1999, Loads 1997, Catching the request for 2000, Loads 1999, Catching the request for 2001, Loads 2000, Catching the request for 2002, Loads 2001, Catching th

,importerCode,importer,HS6,year,preference,tariff_pct
0,124,Canada,010391,1998,MFN,0.0
1,124,Canada,010593,1998,MFN,87.4
2,124,Canada,010599,1998,MFN,56.5
3,124,Canada,020120,1998,MFN,14.0
4,124,Canada,020220,1998,MFN,14.0



Started European Union for MFN
Catching the request for 1980, Catching the request for 1981, Catching the request for 1982, Loads 1982, Catching the request for 1983, Loads 1981, Catching the request for 1984, Loads 1983, Catching the request for 1985, Loads 1984, Catching the request for 1986, Loads 1980, Catching the request for 1987, Loads 1985, Catching the request for 1988, Loads 1986, Catching the request for 1989, Loads 1988Loads 1989, Catching the request for 1990, , Catching the request for 1991, Loads 1990Loads 1991, Catching the request for 1992, , Catching the request for 1993, Loads 1987, Catching the request for 1994, Loads 1993, Catching the request for 1995, Loads 1992, Catching the request for 1996, Loads 1994, Catching the request for 1997, Loads 1996, Catching the request for 1998, Loads 1997, Catching the request for 1999, Loads 1998, Catching the request for 2000, Loads 1999, Catching the request for 2001, Loads 1995, Catching the request for 2002, Loads 2000, Cat

...

,importerCode,importer,exporterCode,exporter,HS6,year,preference,tariff_pct
0,756,Switzerland,784,United Arab Emirates,900830,1996,Generalized System of Preferences (GSP) scheme...,0.0
1,756,Switzerland,788,Tunisia,900830,1996,Generalized System of Preferences (GSP) scheme...,0.0
2,756,Switzerland,792,Turkey,900830,1996,Taux du droit/Accord de libre-�change avec Tur...,0.0
3,756,Switzerland,800,Uganda,900830,1996,Taux du droit/Pays en d�veoppement les moins a...,0.0
4,756,Switzerland,807,North Macedonia,900830,1996,Generalized System of Preferences (GSP) scheme...,0.0



Started Turkey for Preferential
Catching the request for 1980Catching the request for 1981, , Catching the request for 1982, Loads 1982Loads 1981, Catching the request for 1983, , Catching the request for 1984, Loads 1983Loads 1984, , Catching the request for 1985Catching the request for 1986, , Loads 1980, Catching the request for 1987, Loads 1985Loads 1986, Catching the request for 1988, , Catching the request for 1989, Loads 1988, Catching the request for 1990, Loads 1989, Catching the request for 1991, Loads 1990, Catching the request for 1992, Loads 1991, Catching the request for 1993, Loads 1987, Catching the request for 1994, Loads 1992, Catching the request for 1995, Loads 1993, Catching the request for 1996, Loads 1995, Catching the request for 1997, Loads 1996, Catching the request for 1998, Loads 1994, Catching the request for 1999, Loads 1997, Catching the request for 2000, Loads 1998, Catching the request for 2001, Loads 2000, Catching the request for 2002, Loads 2001, Ca

,importerCode,importer,exporterCode,exporter,HS6,year,preference,tariff_pct
0,792,Turkey,276,Germany,840120,1999,EC and Customs Union,0.0
1,792,Turkey,300,Greece,840120,1999,EC and Customs Union,0.0
2,792,Turkey,352,Iceland,840120,1999,EFTA and Free Trade Agreement,0.0
3,792,Turkey,372,Ireland,840120,1999,EC and Customs Union,0.0
4,792,Turkey,376,Israel,840120,1999,FTA with Israel,0.0



Started United States for Preferential
Catching the request for 1980, Catching the request for 1981, Catching the request for 1982, Loads 1980, Catching the request for 1983, Loads 1982, Catching the request for 1984, Loads 1983, Catching the request for 1985, Loads 1984, Catching the request for 1986, Loads 1985, Catching the request for 1987, Loads 1986, Catching the request for 1988, Loads 1981, Catching the request for 1989, Loads 1988, Catching the request for 1990, Loads 1989, Catching the request for 1991, Loads 1990, Catching the request for 1992, Loads 1991, Catching the request for 1993, Loads 1987, Catching the request for 1994, Loads 1992, Catching the request for 1995, Loads 1993, Catching the request for 1996, Loads 1996, Catching the request for 1997, Loads 1994, Catching the request for 1998, Loads 1997, Catching the request for 1999, Loads 1998, Catching the request for 2000, Loads 1999, Catching the request for 2001, Loads 2000, Catching the request for 2002, Loads 1

,importerCode,importer,exporterCode,exporter,HS6,year,preference,tariff_pct
0,840,United States of America,203,Czech Republic,903090,2001,Generalized System of Preferences (GSP) scheme,0.0
1,840,United States of America,204,Benin,903090,2001,Generalized System of Preferences (GSP) scheme...,0.0
2,840,United States of America,212,Dominica,903090,2001,Preferential duty rate for the Caribbean Basin...,0.0
3,840,United States of America,214,Dominican Republic,903090,2001,Preferential duty rate for the Caribbean Basin...,0.0
4,840,United States of America,218,Ecuador,903090,2001,Preferential duty rate under the Andean Trade ...,0.0
